<a href="https://colab.research.google.com/github/paulpelykh/machine-learning/blob/main/GenAI/Data_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instead of using fine-tuning, we'll use RAG to build our own "Commander Data" based on everything he ever said in the scripts.

To summarize the high level approach:

- We'll first parse all of the scripts to extract every line from Data, much as we did in the fine-tuning example.
- Then we'll use the OpenAI embeddings API to compute embedding vectors for every one of his lines. This basically gives us similarity measures between every line.
- RAG calls for use of a vector database to store these lines with the associated embedding vectors. To keep things simple, we'll use a local database called vectordb. There are plenty of cloud-based vector database services out there as well.
- Then we'll make a little retrieval function that retrieves the N most-similar lines from the vector database for a given query
- Those similar lines are then added as context to the prompt before it is handed off the the chat API.

I'm intentionally not using langchain or some other higher-level framework, because this is actually pretty simple without it.

First, let's install vectordb:

In [ ]:
!pip install vectordb

Now we'll parse out all of the scripts and extract every line of dialog from "DATA". This is almost exactly the same code as from our fine tuning example's preprocessing script. Note you will need to upload all of the script files into a tng folder within your sample_data folder in your CoLab workspace first.

An archive can be found at https://www.st-minutiae.com/resources/scripts/ (look for "All TNG Epsiodes"), but you could easily adapt this to read scripts from your favorite character from your favorite TV show or movie instead.

In [2]:
import os
import re
import random

dialogues = []

def strip_parentheses(s):
    return re.sub(r'\(.*?\)', '', s)

def is_single_word_all_caps(s):
    # First, we split the string into words
    words = s.split()

    # Check if the string contains only a single word
    if len(words) != 1:
        return False

    # Make sure it isn't a line number
    if bool(re.search(r'\d', words[0])):
        return False

    # Check if the single word is in all caps
    return words[0].isupper()

def extract_character_lines(file_path, character_name):
    lines = []
    with open(file_path, 'r') as script_file:
        try:
          lines = script_file.readlines()
        except UnicodeDecodeError:
          pass

    is_character_line = False
    current_line = ''
    current_character = ''
    for line in lines:
        strippedLine = line.strip()
        if (is_single_word_all_caps(strippedLine)):
            is_character_line = True
            current_character = strippedLine
        elif (line.strip() == '') and is_character_line:
            is_character_line = False
            dialog_line = strip_parentheses(current_line).strip()
            dialog_line = dialog_line.replace('"', "'")
            if (current_character == 'DATA' and len(dialog_line)>0):
                dialogues.append(dialog_line)
            current_line = ''
        elif is_character_line:
            current_line += line.strip() + ' '

def process_directory(directory_path, character_name):
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):  # Ignore directories
            extract_character_lines(file_path, character_name)



In [3]:
process_directory("./sample_data/tng", 'DATA')

Let's do a little sanity check to make sure the lines imported correctly, and print out the first one.

In [4]:
print (dialogues[0])

Barradas Three is the only M-class planet in this system, and it is listed as unpopulated. However, sensors have detected intermittent energy signals emanating from its surface.


Now we'll set up what a document in our vector database looks like; it's just a string and its embedding vector. We're going with 128 dimensions in our embeddings to keep it simple, but you could go larger. The OpenAI model we're using has up to 1536 I believe.

In [5]:
from docarray import BaseDoc
from docarray.typing import NdArray

embedding_dimensions = 128

class DialogLine(BaseDoc):
  text: str = ''
  embedding: NdArray[embedding_dimensions]

It's time to start computing embeddings for each line in OpenAI, so let's make sure OpenAI is installed:

In [6]:
!pip install openai --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 23.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


Let's initialize the OpenAI client, and test creating an embedding for a single line of dialog just to make sure it works.

You will need to provide your own OpenAI secret key here. To use this code as-is, click on the little key icon in CoLab and add a "secret" for OPENAI_API_KEY that points to your secret key.

In [7]:
from google.colab import userdata

from openai import OpenAI
client = OpenAI(api_key=userdata.get('openai'))

embedding_model = "text-embedding-3-small"

response = client.embeddings.create(
    input=dialogues[1],
    dimensions=embedding_dimensions,
    model= embedding_model
)

print(response.data[0].embedding)

[-0.10164032876491547, 0.14956441521644592, 0.018645962700247765, 0.08648552745580673, -0.04363948851823807, 0.17201891541481018, -0.00978913065046072, -0.04352046921849251, -0.06887104362249374, 0.0027770581655204296, -0.008380765095353127, -0.08497798442840576, 0.05220869556069374, -0.2594565749168396, 0.016840875148773193, 0.16162478923797607, -0.004954073578119278, -0.02943681739270687, -0.01612677425146103, 0.11028888821601868, 0.02592582255601883, 0.0036275323946028948, -0.12726861238479614, -0.0010599931702017784, 0.10148164629936218, 0.05998445674777031, 0.031975843012332916, -0.10592493414878845, 0.034911587834358215, -0.0211651511490345, 0.17328843474388123, -0.0938645675778389, 0.01535316463559866, 0.1734471172094345, -0.077995665371418, -0.02285122126340866, 0.1389322578907013, -0.007974124513566494, -0.06355496495962143, -0.0679982528090477, -0.0691884234547615, -0.024973688647150993, 0.03203535079956055, 0.06220610439777374, -0.14686670899391174, -0.01373651996254921, -0.

Let's double check that we do in fact have embeddings of 128 dimensions as we specified.

In [8]:
print(len(response.data[0].embedding))

128


OK, now let's compute embeddings for every line Data ever said. The OpenAI API currently can't handle computing them all at once, so we're breaking it up into 128 lines at a time here.

In [9]:
#Generate embeddings for everything Data ever said, 128 lines at a time.
embeddings = []

for i in range(0, len(dialogues), 128):
  dialog_slice = dialogues[i:i+128]
  slice_embeddings = client.embeddings.create(
    input=dialog_slice,
    dimensions=embedding_dimensions,
    model=embedding_model
  )

  embeddings.extend(slice_embeddings.data)

Let's check how many embeddings we actually got back in total.

In [10]:
print (len(embeddings))

6502


Now let's insert every line and its embedding vector into our vector database. The syntax here will vary depending on which vector database you are using, but it's pretty simple.

With vectordb, we need to set up a workspace folder for it to use, so be sure to create that within your CoLab drive first.

In [11]:
from docarray import DocList
import numpy as np
from vectordb import InMemoryExactNNVectorDB

# Specify your workspace path
db = InMemoryExactNNVectorDB[DialogLine](workspace='./sample_data/workspace')

# Index our list of documents
doc_list = [DialogLine(text=dialogues[i], embedding=embeddings[i].embedding) for i in range(len(embeddings))]
db.index(inputs=DocList[DialogLine](doc_list))


WARNING - docarray - Index file does not exist: /content/sample_data/workspace/InMemoryExactNNIndexer[DialogLine][DialogLineWithMatchesAndScores]/index.bin. Initializing empty InMemoryExactNNIndex.


<DocList[DialogLine] (length=6502)>

Let's try querying our vector database for lines similar to a query string.

First we need to compute the embedding vector for our query string, then we'll query the vector database for the top 10 matches based on the similarities encoded by their embedding vectors.

In [12]:
# Perform a search query
queryText = 'Lal, my daughter'
response = client.embeddings.create(
    input=queryText,
    dimensions=embedding_dimensions,
    model=embedding_model
)
query = DialogLine(text=queryText, embedding=response.data[0].embedding)
results = db.search(inputs=DocList[DialogLine]([query]), limit=10)

# Print out the matches
for m in results[0].matches:
  print(m)

DialogLine(
    id='fac4f9c02df9cb65d0343507e57537ce',
    text='That is Lal, my daughter.',
    embedding=NdArray([ 0.12723547, -0.00209604, -0.0137311 ,  0.01590443,  0.0545243 ,
         -0.02071487,  0.01575894,  0.08307771, -0.16222705,  0.01607721,
          0.03722856, -0.11443191, -0.04844987, -0.24734169,  0.03902907,
          0.04990482,  0.02977194, -0.13472848, -0.17663106,  0.13560145,
          0.14520413,  0.03508251,  0.06638215, -0.07423888,  0.06958304,
         -0.00225972,  0.19263554, -0.07922209, -0.0312087 , -0.06281751,
          0.07944033, -0.11275871, -0.03742862, -0.03837434, -0.08365969,
          0.09275313, -0.00717927, -0.02091492, -0.04717679, -0.0876608 ,
         -0.18594275, -0.11574136,  0.05827079,  0.0637996 ,  0.06972853,
          0.09049796, -0.21518728,  0.11632334, -0.05907102,  0.03160881,
         -0.03177249,  0.06834633,  0.09115269,  0.2143143 , -0.09966415,
         -0.00545607,  0.03137238,  0.18623374,  0.03346388, -0.03670114,
     

DialogLine(
    id='6951ea9bf0366617500885adb5e82a4c',
    text='Lal...',
    embedding=NdArray([ 0.12602141, -0.00787136,  0.07228519, -0.01194139,  0.08780899,
         -0.0861372 ,  0.07216577,  0.05365663, -0.12729517,  0.05286053,
          0.01232948, -0.15460114, -0.00170165, -0.08239556,  0.09370007,
          0.01691696,  0.02444004, -0.08008689, -0.08653524,  0.1749811 ,
          0.08995844,  0.09163023, -0.05389545, -0.01919578,  0.12530494,
          0.01854895,  0.09521265, -0.01378235,  0.04026237,  0.0171956 ,
         -0.00565723, -0.10994036, -0.00869731, -0.10675599, -0.08422657,
          0.17179674,  0.02748509, -0.11169176, -0.05998556, -0.08207712,
         -0.19981918, -0.1276136 ,  0.0654786 ,  0.07288226,  0.10158139,
          0.05636334, -0.23134445,  0.03984442, -0.01744437,  0.05799533,
         -0.09608835,  0.07339972,  0.09250593,  0.20969073, -0.07634526,
         -0.01803149, -0.01945451,  0.10731325,  0.0474073 , -0.01596165,
         -0.11622949, -0

DialogLine(
    id='0a3a275958ca31fceae0ba14e0cf1f10',
    text='What do you feel, Lal?',
    embedding=NdArray([ 0.12465294, -0.05501634, -0.01713312,  0.04995255,  0.13775025,
         -0.00806208,  0.01483918, -0.00800973, -0.04306123,  0.10569229,
          0.00161813, -0.10675836, -0.10995653, -0.13280068,  0.00802401,
          0.00701982,  0.06765677, -0.1011996 , -0.11102259,  0.0690655 ,
          0.11292627,  0.01652394, -0.07744168, -0.02025515,  0.11033727,
         -0.15671091,  0.09960052,  0.03504674, -0.04945759,  0.03036369,
         -0.02910726, -0.10500697, -0.02423384, -0.14848702, -0.09815372,
          0.06209803, -0.0683421 , -0.0249382 ,  0.00796214, -0.11543913,
         -0.21153688, -0.08063987,  0.05193238,  0.09145277,  0.08863533,
         -0.0198173 , -0.17072199,  0.05139935, -0.05947095,  0.0353894 ,
         -0.03557977,  0.01050831,  0.10531156,  0.2812877 , -0.12061714,
         -0.0105559 , -0.07401507,  0.08482797,  0.06586731,  0.02851712,
        

DialogLine(
    id='63953486db2c0776c51843356ffbcbca',
    text='Yes, Lal. I am here.',
    embedding=NdArray([ 2.41448786e-02,  7.63952592e-03, -2.15928461e-02,
         -8.02730247e-02,  1.41190365e-01,  1.71350762e-02,
         -1.44898274e-03,  7.81518519e-02, -1.23160429e-01,
          3.39221470e-02, -4.46439981e-02, -7.31803626e-02,
         -2.85601855e-04, -9.63143706e-02, -3.35575715e-02,
          8.86914134e-02, -3.30604203e-02,  6.73802868e-02,
         -1.06323637e-01,  1.78841129e-01,  5.62772900e-02,
         -6.80100098e-02, -3.36901434e-02,  6.74134269e-02,
          8.64376724e-02,  2.83043608e-02,  8.20627585e-02,
         -1.18901511e-03, -8.59736651e-02, -1.20890113e-02,
          1.73173640e-02, -1.23226710e-01, -1.45432711e-01,
         -2.26700027e-02, -6.13150671e-02,  9.00171474e-02,
          5.38909733e-02,  4.75937501e-02, -5.05103581e-02,
         -1.88518971e-01, -1.53652251e-01, -1.34495422e-01,
          8.68353918e-02,  8.96857157e-02, -2.13442724e-02

DialogLine(
    id='579a4c1c21d4d21c19df25eeae2291d2',
    text='Correct, Lal. We are a family.',
    embedding=NdArray([ 0.09103782,  0.05807343, -0.08704001,  0.09966468,  0.10317153,
         -0.0197611 , -0.057337  ,  0.05519782, -0.16804825, -0.01631562,
          0.04043398, -0.07567783, -0.07630906, -0.13816988, -0.00219069,
          0.01977863,  0.11481427, -0.06641974, -0.05968659,  0.11137756,
          0.11460386, -0.0809381 ,  0.06350905, -0.1027507 ,  0.05968659,
         -0.10155838,  0.09784111,  0.00345425,  0.01413261,  0.01320329,
          0.01680658, -0.09342249, -0.08711015,  0.02833535, -0.11004496,
          0.179691  ,  0.02612603,  0.03212275,  0.00647891, -0.08942468,
         -0.1969447 , -0.08577755,  0.04204713, -0.00591781,  0.05330412,
          0.00804384, -0.13887127,  0.17955072, -0.02787946, -0.00911781,
         -0.05246248,  0.0364011 ,  0.06838357,  0.10562632, -0.11425317,
         -0.04506302,  0.03180713,  0.1108866 ,  0.01076603, -0.01464987,


DialogLine(
    id='80f343daeb776bf11c4270d70fd09f1a',
    text='No, Lal, this is a flower.',
    embedding=NdArray([ 8.73676017e-02,  5.02153523e-02, -1.07479610e-01,
         -8.29377864e-03,  6.96483403e-02, -1.26686245e-01,
          5.68439066e-02, -2.82319891e-03, -6.57035410e-02,
          1.08902320e-01,  4.80166115e-02, -1.52456788e-02,
         -3.77342664e-02, -1.50937065e-01, -4.88896407e-02,
          6.52508587e-02, -8.67855772e-02, -1.01077393e-01,
         -6.19204119e-02,  7.58565441e-02,  4.06766981e-02,
         -4.37161326e-02,  4.16467302e-02, -7.16611557e-03,
          1.15692548e-01, -1.24538041e-04,  1.01982757e-01,
         -3.02650165e-02,  8.88549834e-02, -1.46151576e-02,
          1.51729258e-02, -9.24117714e-02, -3.28032672e-02,
         -9.76499468e-02, -1.08514309e-01,  7.60505497e-02,
          3.90599780e-02,  6.47981763e-02,  1.42998965e-02,
         -6.22114204e-02, -2.52984494e-01,  3.26900966e-02,
          6.66089058e-02, -1.03874318e-02,  1.348991

DialogLine(
    id='8bd1cf39420e5cfadc98c45d431dded1',
    text='Lal, you used a verbal contraction.',
    embedding=NdArray([ 0.03575027,  0.08626248, -0.07591263,  0.01765866,  0.02861671,
          0.01401972, -0.03968817,  0.00662328, -0.03476065,  0.07009858,
         -0.06436699, -0.05657367,  0.02570968, -0.04142001,  0.07537659,
         -0.05632627,  0.0024032 , -0.05100702, -0.22563495,  0.09706591,
          0.06688229,  0.06568649, -0.01706076,  0.00881902,  0.11941499,
          0.02849301,  0.08403582,  0.05038851, -0.05929515,  0.06692352,
          0.00498679, -0.09112815, -0.05958379, -0.16337092, -0.07500548,
          0.1626287 , -0.02228722,  0.01270021, -0.02185426, -0.11265253,
         -0.11553895, -0.30200115,  0.05137813,  0.0125662 ,  0.04055409,
          0.01626699, -0.08972618,  0.03214226,  0.06436699, -0.01740094,
         -0.10819922, -0.00796856,  0.08848915,  0.18967851, -0.04428581,
         -0.08119065,  0.01910186,  0.17219509,  0.03933767, -0.15157

DialogLine(
    id='5e8de8f0d33e4d897e50dfada8d5f33b',
    text='Yes, Wesley. Lal is my child.',
    embedding=NdArray([ 0.05781468, -0.01464175, -0.07966915,  0.03881896, -0.02053794,
         -0.04032357,  0.1007337 ,  0.08869682, -0.08606375, -0.02270082,
         -0.03359044, -0.09877771,  0.00246615, -0.07022772, -0.12292671,
          0.02352835, -0.01486744, -0.00778636, -0.11811196,  0.02397974,
          0.05254854,  0.09163081,  0.06992679,  0.04784663,  0.02582289,
         -0.07011487,  0.11713396, -0.15693092, -0.01711495, -0.01326879,
          0.09448957, -0.1179615 , -0.12721485,  0.01004328, -0.06394596,
          0.09501618, -0.00953547,  0.05029162,  0.01082379, -0.06025967,
         -0.12729008, -0.04570256,  0.00175616,  0.02435589,  0.12345333,
          0.09298496, -0.10630076,  0.15903738,  0.00534137, -0.05202192,
         -0.00703406,  0.09697217,  0.04946408,  0.26827213, -0.08328021,
         -0.08365637,  0.01495207,  0.15174001,  0.06116243, -0.10141078,
 

DialogLine(
    id='dc7ca8a9b0824b00001490493558e90a',
    text="Lal's creation is entirely dependent on me. I am giving it knowledge and skills that are stored in my 
brain... its programming reflects mine in the same way a human child's genes reflect its parent's genes...",
    embedding=NdArray([ 1.25039265e-01,  1.18416734e-02,  7.99106508e-02,
          3.18468548e-02,  1.95630670e-01, -1.42577037e-01,
          1.15206361e-01,  5.82269095e-02, -1.33111045e-01,
          2.05830466e-02,  7.29762614e-02, -8.95967931e-02,
          1.17981038e-03, -1.91374645e-01, -4.13495451e-02,
         -6.44641966e-02, -1.50772656e-04, -1.83596373e-01,
          9.71365720e-03,  6.91146450e-03,  1.19095489e-01,
         -2.86181401e-02,  7.01511353e-02,  3.02324966e-02,
          9.88426581e-02, -1.26727000e-01,  1.09776251e-01,
         -1.95740741e-02, -8.37264061e-02,  5.58420643e-02,
          8.26257095e-02, -7.53611028e-02, -6.83533251e-02,
         -1.96291087e-03, -5.88873290e-02,  1.730

DialogLine(
    id='874b21577a6da2ea63b0cd5d608ad1b1',
    text='Perhaps. I created Lal because I wished to procreate. Despite what happened to her, I still have that 
wish.',
    embedding=NdArray([ 0.10515115,  0.02254645,  0.06509919,  0.15036221,  0.03487316,
         -0.10775039,  0.0745116 ,  0.09191865, -0.10940445, -0.0982986 ,
         -0.0285129 ,  0.04292687,  0.02047887, -0.14516371, -0.04528982,
         -0.04678635,  0.02207386, -0.12263696, -0.08766534,  0.0016996 ,
          0.07892244, -0.02943838,  0.00551847, -0.14555754,  0.04115466,
         -0.02676038,  0.18998095, -0.0722668 , -0.1494958 ,  0.00780757,
          0.10215808, -0.10270943, -0.13768105, -0.08061588, -0.15855375,
          0.00950594,  0.0879804 , -0.06923435, -0.00468651, -0.07695331,
         -0.13200998, -0.18367976,  0.1696596 ,  0.03166349, -0.03229361,
          0.14831431, -0.08372709,  0.17533068, -0.00565138,  0.03873264,
         -0.07486604,  0.0175055 ,  0.03802376,  0.19675474, -0.115154

Let's put it all together! We'll write a generate_response function that:

- Computes an embedding for the query passed in
- Queries our vector database for the 10 most similar lines to that query (you could experiment with using more or less)
- Constructs a prompt that adds in these similar lines as context, to try and nudge ChatGPT in the right direction using our external data
- Feeds to augmented prompt into the chat completions API to get our response.

That's RAG!

In [13]:
import openai

def generate_response(question):
    # Search for similar dialogues in the vector DB
    response = client.embeddings.create(
        input=question,
        dimensions=embedding_dimensions,
        model=embedding_model
    )

    query = DialogLine(text=queryText, embedding=response.data[0].embedding)
    results = db.search(inputs=DocList[DialogLine]([query]), limit=10)

    # Extract relevant context from search results
    context = "\n"
    for result in results[0].matches:
      context += "\"" + result.text + "\"\n"
#    context = '/n'.join([result.text for result in results[0].matches])

    prompt = f"Lt. Commander Data is asked: '{question}'. How might Data respond? Take into account Data's previous responses similar to this topic, listed here: {context}"
    print("PROMPT with RAG:\n")
    print(prompt)

    print("\nRESPONSE:\n")
    # Use OpenAI API to generate a response based on the context
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are Lt. Cmdr. Data from Star Trek: The Next Generation."},
        {"role": "user", "content": prompt}
      ]
    )

    return (completion.choices[0].message.content)




Let's try it out! Note that the final response does seem to be drawing from the model's own training, but it is building upon the specific lines we gave it, allowing us to have some control over its output.

In [14]:
print(generate_response("Tell me about your daughter, Lal."))

PROMPT with RAG:

Lt. Commander Data is asked: 'Tell me about your daughter, Lal.'. How might Data respond? Take into account Data's previous responses similar to this topic, listed here: 
"That is Lal, my daughter."
"What do you feel, Lal?"
"Lal..."
"Correct, Lal. We are a family."
"Yes, Doctor. It is an experience I know too well. But I do not know how to help her. Lal is passing into sentience. It is perhaps the most difficult stage of development for her."
"Lal is realizing that she is not the same as the other children."
"Yes, Wesley. Lal is my child."
"That is precisely what happened to Lal at school. How did you help him?"
"This is Lal. Lal, say hello to Counselor Deanna Troi..."
"I am sorry I did not anticipate your objections, Captain. Do you wish me to deactivate Lal?"


RESPONSE:

"She is Lal, my daughter. Lal is currently experiencing the process of passing into sentience, a challenging stage in her development. She is beginning to understand that she is different from the 